In [1]:
!pip install codecarbon

In [2]:
!pip install thop

In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim.lr_scheduler import ReduceLROnPlateau
from torch.utils.data import DataLoader, random_split
from torchvision import models, datasets, transforms
import time
import os
import subprocess
import numpy as np
from tqdm import tqdm
import codecarbon
from thop import profile

# Check if GPU is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Define the transforms for the dataset with additional augmentations
transform_train = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomCrop(32, padding=4),
    transforms.RandomRotation(10),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

# Load the CIFAR-10 dataset
full_train_dataset = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform_train)
test_dataset = datasets.CIFAR10(root='./data', train=False, download=True, transform=transform_test)

# Split the full training dataset into training and validation sets (80% train, 20% validation)
train_size = int(0.8 * len(full_train_dataset))
val_size = len(full_train_dataset) - train_size
train_dataset, val_dataset = random_split(full_train_dataset, [train_size, val_size])

# Create DataLoaders for the train, validation, and test sets
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True, num_workers=4, pin_memory=True)
val_loader = DataLoader(val_dataset, batch_size=64, shuffle=False, num_workers=4, pin_memory=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False, num_workers=4, pin_memory=True)

# Print dataset sizes for debugging
print(f"Training set size: {len(train_dataset)}")
print(f"Validation set size: {len(val_dataset)}")
print(f"Test set size: {len(test_dataset)}")

# Load the pretrained VGG16 model and modify the classifier
model = models.vgg16(pretrained=True)
input_lastLayer = model.classifier[6].in_features
model.classifier[6] = nn.Sequential(
    nn.Dropout(0.5),  # Increased dropout for regularization
    nn.Linear(input_lastLayer, 10)
)
model = model.to(device)

# Define loss function and optimizer with learning rate scheduler
learning_rate = 0.01
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=learning_rate, momentum=0.9, weight_decay=5e-4)
scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=3, verbose=True)

# Tracking metrics for analysis
train_losses, val_losses = [], []
train_accuracies, val_accuracies = [], []

# Use CodeCarbon for system-wide energy consumption tracking
tracker = codecarbon.EmissionsTracker()
tracker.start()

# Function to get GPU power consumption using nvidia-smi
def get_gpu_power():
    try:
        # Query power.draw from nvidia-smi
        result = subprocess.run(
            ['nvidia-smi', '--query-gpu=power.draw', '--format=csv,nounits,noheader'],
            stdout=subprocess.PIPE,
            stderr=subprocess.PIPE,
            text=True
        )
        power_draws = result.stdout.strip().split('\n')
        power_draws = [float(p) for p in power_draws]
        avg_power_draw = sum(power_draws) / len(power_draws)  # Average power across GPUs
        return avg_power_draw  # in watts
    except Exception as e:
        print(f"Error getting GPU power: {e}")
        return 0.0

# Tracking GPU-specific energy consumption
gpu_power_readings = []

# Training loop with GPU power monitoring
epochs = 30
start_training_time = time.time()

for epoch in range(epochs):
    # Training phase
    model.train()
    running_loss = 0.0
    correct_train = 0
    total_train = 0
    for images, labels in tqdm(train_loader, desc=f"Epoch {epoch+1}/{epochs} - Training"):
        images, labels = images.to(device), labels.to(device)

        # Zero the parameter gradients
        optimizer.zero_grad()

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backward pass and optimize
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

        # Calculate training accuracy
        _, predicted = torch.max(outputs.data, 1)
        total_train += labels.size(0)
        correct_train += (predicted == labels).sum().item()

        # Record GPU power usage
        gpu_power_readings.append(get_gpu_power())

    train_loss = running_loss / len(train_loader)
    train_accuracy = 100 * correct_train / total_train

    # Append metrics for plotting
    train_losses.append(train_loss)
    train_accuracies.append(train_accuracy)

    # Validation phase
    model.eval()
    running_val_loss = 0.0
    correct_val = 0
    total_val = 0
    with torch.no_grad():
        for images, labels in tqdm(val_loader, desc=f"Epoch {epoch+1}/{epochs} - Validation"):
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)

            running_val_loss += loss.item()

            # Calculate validation accuracy
            _, predicted = torch.max(outputs.data, 1)
            total_val += labels.size(0)
            correct_val += (predicted == labels).sum().item()

    val_loss = running_val_loss / len(val_loader)
    val_accuracy = 100 * correct_val / total_val

    # Append metrics for plotting
    val_losses.append(val_loss)
    val_accuracies.append(val_accuracy)

    print(f"Epoch [{epoch+1}/{epochs}], Train Loss: {train_loss:.4f}, Train Accuracy: {train_accuracy:.2f}%")
    print(f"Epoch [{epoch+1}/{epochs}], Val Loss: {val_loss:.4f}, Val Accuracy: {val_accuracy:.2f}%")

    # Adjust learning rate based on validation loss
    scheduler.step(val_loss)

training_time = time.time() - start_training_time

# Stop the CodeCarbon tracker after training
emissions = tracker.stop()

# Calculate average GPU power and total GPU energy consumption (in kWh)
avg_gpu_power = sum(gpu_power_readings) / len(gpu_power_readings) if gpu_power_readings else 0.0
energy_consumption_gpu = (avg_gpu_power * training_time) / 3600  # Convert to kWh

# Save the model after training
model_path = "trained_vgg16_model.pth"
torch.save(model.state_dict(), model_path)

# Model analysis metrics
param_count = sum(p.numel() for p in model.parameters() if p.requires_grad)
torch.save(model.state_dict(), "temp_model.pth")
model_size = os.path.getsize("temp_model.pth") / (1024 * 1024)  # Convert bytes to MB
os.remove("temp_model.pth")

# FLOPs calculation
dummy_input = torch.randn(1, 3, 32, 32).to(device)
flops, _ = profile(model, inputs=(dummy_input,), verbose=False)

# Inference time calculation
model.eval()
inference_start = time.time()
with torch.no_grad():
    for _ in range(100):
        _ = model(dummy_input)
inference_time = (time.time() - inference_start) / 100

# Final energy and emissions report
print("\n--- Model Analysis ---")
print(f"Parameter Count: {param_count}")
print(f"Model Size: {model_size:.2f} MB")
print(f"FLOPs: {flops / 1e9:.2f} GFLOPs")
print(f"Training Time: {training_time:.2f} seconds")
print(f"Average Inference Time: {inference_time:.6f} seconds")

print("\n--- Energy and Emissions Report ---")
print(f"CO2 Emissions (CodeCarbon): {emissions:.6f} kg")
print(f"Average GPU Power Consumption: {avg_gpu_power:.2f} W")
print(f"Total GPU Energy Consumption: {energy_consumption_gpu:.6f} kWh")


Using device: cuda


100%|██████████| 170M/170M [00:05<00:00, 29.8MB/s]


Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:617: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_We

Training set size: 40000
Validation set size: 10000
Test set size: 10000


...

Epoch [28/30], Train Loss: 0.1188, Train Accuracy: 95.89%
Epoch [28/30], Val Loss: 0.3258, Val Accuracy: 89.71%


Epoch 29/30 - Training:   0%|          | 0/625 [00:00<?, ?it/s][codecarbon INFO @ 21:56:31] Energy consumed for RAM : 0.002770 kWh. RAM Power : 4.753036022186279 W
[codecarbon INFO @ 21:56:31] Energy consumed for all GPUs : 0.035502 kWh. Total GPU Power : 49.5397217325232 W
[codecarbon INFO @ 21:56:31] Energy consumed for all CPUs : 0.024799 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 21:56:31] 0.063071 kWh of electricity used since the beginning.
Epoch 29/30 - Training:  23%|██▎       | 146/625 [00:15<00:43, 11.01it/s][codecarbon INFO @ 21:56:46] Energy consumed for RAM : 0.002790 kWh. RAM Power : 4.753036022186279 W
[codecarbon INFO @ 21:56:46] Energy consumed for all GPUs : 0.035764 kWh. Total GPU Power : 62.80500163678455 W
[codecarbon INFO @ 21:56:46] Energy consumed for all CPUs : 0.024976 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 21:56:46] 0.063530 kWh of electricity used since the beginning.
Epoch 29/30 - Training:  47%|████▋     | 294/625 [00:30<00:29, 11.12it/s][c

Epoch [29/30], Train Loss: 0.1124, Train Accuracy: 96.23%
Epoch [29/30], Val Loss: 0.3297, Val Accuracy: 89.81%


Epoch 30/30 - Training:   2%|▏         | 10/625 [00:01<01:02,  9.90it/s][codecarbon INFO @ 21:57:46] Energy consumed for RAM : 0.002869 kWh. RAM Power : 4.753036022186279 W
[codecarbon INFO @ 21:57:46] Energy consumed for all GPUs : 0.036778 kWh. Total GPU Power : 48.21986379037768 W
[codecarbon INFO @ 21:57:46] Energy consumed for all CPUs : 0.025685 kWh. Total CPU Power : 42.5 W
Epoch 30/30 - Training:  26%|██▌       | 160/625 [00:16<00:42, 10.86it/s][codecarbon INFO @ 21:58:01] Energy consumed for RAM : 0.002889 kWh. RAM Power : 4.753036022186279 W
[codecarbon INFO @ 21:58:01] Energy consumed for all GPUs : 0.037046 kWh. Total GPU Power : 64.2501436318214 W
[codecarbon INFO @ 21:58:01] Energy consumed for all CPUs : 0.025862 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 21:58:01] 0.065797 kWh of electricity used since the beginning.
Epoch 30/30 - Training:  50%|█████     | 313/625 [00:31<00:28, 11.04it/s][codecarbon INFO @ 21:58:16] Energy consumed for RAM : 0.002909 kWh. RAM Pow

Epoch [30/30], Train Loss: 0.1100, Train Accuracy: 96.37%
Epoch [30/30], Val Loss: 0.3232, Val Accuracy: 90.37%

--- Model Analysis ---
Parameter Count: 134301514
Model Size: 512.33 MB
FLOPs: 0.43 GFLOPs
Training Time: 2249.39 seconds
Average Inference Time: 0.003655 seconds

--- Energy and Emissions Report ---
CO2 Emissions (CodeCarbon): 0.018069 kg
Average GPU Power Consumption: 64.05 W
Total GPU Energy Consumption: 40.018244 kWh


In [6]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim.lr_scheduler import CosineAnnealingLR
from torchvision import models, datasets, transforms
from torch.utils.data import DataLoader, random_split
from thop import profile
from codecarbon import EmissionsTracker
import time
import os
import subprocess

# Check if GPU is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Define the transforms for the dataset
transform_train = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomCrop(32, padding=4),
    transforms.RandomRotation(10),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

# Load the CIFAR-10 dataset
full_train_dataset = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform_train)
test_dataset = datasets.CIFAR10(root='./data', train=False, download=True, transform=transform_test)

# Split the full training dataset into training and validation sets
train_size = int(0.8 * len(full_train_dataset))
val_size = len(full_train_dataset) - train_size
train_dataset, val_dataset = random_split(full_train_dataset, [train_size, val_size])

# Create DataLoaders for the train, validation, and test sets
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True, num_workers=4, pin_memory=True)
val_loader = DataLoader(val_dataset, batch_size=64, shuffle=False, num_workers=4, pin_memory=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False, num_workers=4, pin_memory=True)

# Define the teacher model (VGG16) and load pre-trained weights
teacher_model = models.vgg16(pretrained=False)
teacher_last_layer = teacher_model.classifier[6].in_features
teacher_model.classifier[6] = nn.Sequential(
    nn.Dropout(0.5),
    nn.Linear(teacher_last_layer, 10)
)
teacher_model.load_state_dict(torch.load("trained_vgg16_model.pth"))  # Load the downloaded teacher model
teacher_model = teacher_model.to(device)
teacher_model.eval()  # Set the teacher model to evaluation mode

# Define the student model (VGG11)
student_model = models.vgg11(pretrained=False)
student_last_layer = student_model.classifier[6].in_features
student_model.classifier[6] = nn.Sequential(
    nn.Dropout(0.5),
    nn.Linear(student_last_layer, 10)
)
student_model = student_model.to(device)

# Define the distillation loss
class DistillationLoss(nn.Module):
    def __init__(self, temperature=3.0, alpha=0.5):
        super(DistillationLoss, self).__init__()
        self.temperature = temperature
        self.alpha = alpha
        self.criterion_ce = nn.CrossEntropyLoss()

    def forward(self, student_logits, teacher_logits, labels):
        distillation_loss = nn.KLDivLoss(reduction='batchmean')(
            torch.log_softmax(student_logits / self.temperature, dim=1),
            torch.softmax(teacher_logits / self.temperature, dim=1)
        ) * (self.temperature ** 2)
        student_loss = self.criterion_ce(student_logits, labels)
        return self.alpha * distillation_loss + (1 - self.alpha) * student_loss

criterion = DistillationLoss(temperature=3.0, alpha=0.5)

# Define optimizer and scheduler
optimizer = optim.Adam(student_model.parameters(), lr=0.001, weight_decay=1e-4)
scheduler = CosineAnnealingLR(optimizer, T_max=10)

# GPU power tracking
def get_gpu_power():
    try:
        result = subprocess.run(
            ['nvidia-smi', '--query-gpu=power.draw', '--format=csv,nounits,noheader'],
            stdout=subprocess.PIPE, text=True
        )
        power_draws = [float(p) for p in result.stdout.strip().split('\n')]
        return sum(power_draws) / len(power_draws) if power_draws else 0.0
    except Exception as e:
        print(f"Error getting GPU power: {e}")
        return 0.0

# Energy tracking
tracker = EmissionsTracker()
tracker.start()

# Training loop
epochs = 30
gpu_power_readings = []
start_training_time = time.time()

for epoch in range(epochs):
    student_model.train()
    running_loss = 0.0
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)

        # Teacher predictions
        with torch.no_grad():
            teacher_logits = teacher_model(inputs)

        # Student predictions and loss computation
        student_logits = student_model(inputs)
        loss = criterion(student_logits, teacher_logits, labels)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        gpu_power_readings.append(get_gpu_power())

    scheduler.step()
    print(f"Epoch [{epoch+1}/{epochs}], Loss: {running_loss / len(train_loader):.4f}")

end_training_time = time.time()
training_time = end_training_time - start_training_time

emissions = tracker.stop()

# Energy metrics
avg_gpu_power = sum(gpu_power_readings) / len(gpu_power_readings) if gpu_power_readings else 0.0
total_energy_consumption = (avg_gpu_power * training_time) / 3600  # kWh

# Model analysis
param_count = sum(p.numel() for p in student_model.parameters() if p.requires_grad)
torch.save(student_model.state_dict(), "student_model_vgg11_temp.pth")
model_size_mb = os.path.getsize("student_model_vgg11_temp.pth") / (1024 * 1024)

# FLOPs and inference time
example_input = torch.randn(1, 3, 32, 32).to(device)
flops, _ = profile(student_model, inputs=(example_input,), verbose=False)

student_model.eval()
with torch.no_grad():
    start_time = torch.cuda.Event(enable_timing=True)
    end_time = torch.cuda.Event(enable_timing=True)
    start_time.record()
    for _ in range(100):
        _ = student_model(example_input)
    end_time.record()
    torch.cuda.synchronize()
inference_time_ms = start_time.elapsed_time(end_time) / 100  # Average in milliseconds

# Output results
print("\n--- Model Analysis ---")
print(f"Parameter Count: {param_count}")
print(f"Model Size: {model_size_mb:.2f} MB")
print(f"FLOPs: {flops / 1e9:.2f} GFLOPs")
print(f"Training Time: {training_time:.2f} seconds")
print(f"Average Inference Time: {inference_time_ms:.6f} ms")

print("\n--- Energy and Emissions Report ---")
print(f"CO2 Emissions (CodeCarbon): {emissions:.6f} kg")
print(f"Average GPU Power Consumption: {avg_gpu_power:.2f} W")
print(f"Total GPU Energy Consumption: {total_energy_consumption:.6f} kWh")

Using device: cuda
Files already downloaded and verified
Files already downloaded and verified


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)
<ipython-input-6-b462679c7e6e>:53: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions

Epoch [1/30], Loss: 6.3950


/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:617: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(
[codecarbon INFO @ 22:02:48] Energy consumed for RAM : 0.000119 kWh. RAM Power : 4.753036022186279 W
[codecarbon INFO @ 22:02:48] Energy consumed for all GPUs : 0.001639 kWh. Total GPU Power : 64.81371435116044 W
[codecarbon INFO @ 22:02:48] Energy consumed for all CPUs : 0.001063 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 22:02:48] 0.002821 kWh of electricity used since the beginning.
[codecarbon INFO @ 22:03:03] Energy consumed for RAM : 0.000139 kWh. RAM Power : 4.753036022186279 W
[codecarbon INFO @ 22:03:03] Energy consumed for 

Epoch [2/30], Loss: 5.4064


...

Epoch [28/30], Loss: 1.1598


/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:617: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(
[codecarbon INFO @ 22:36:19] Energy consumed for RAM : 0.002771 kWh. RAM Power : 4.753036022186279 W
[codecarbon INFO @ 22:36:19] Energy consumed for all GPUs : 0.038375 kWh. Total GPU Power : 65.22346268957816 W
[codecarbon INFO @ 22:36:19] Energy consumed for all CPUs : 0.024800 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 22:36:19] 0.065946 kWh of electricity used since the beginning.
[codecarbon INFO @ 22:36:34] Energy consumed for RAM : 0.002791 kWh. RAM Power : 4.753036022186279 W
[codecarbon INFO @ 22:36:34] Energy consumed for 

Epoch [29/30], Loss: 1.0893


/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:617: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(
[codecarbon INFO @ 22:37:19] Energy consumed for RAM : 0.002850 kWh. RAM Power : 4.753036022186279 W
[codecarbon INFO @ 22:37:19] Energy consumed for all GPUs : 0.039469 kWh. Total GPU Power : 65.37872001713211 W
[codecarbon INFO @ 22:37:19] Energy consumed for all CPUs : 0.025508 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 22:37:19] 0.067828 kWh of electricity used since the beginning.
[codecarbon INFO @ 22:37:19] 0.008392 g.CO2eq/s mean an estimation of 264.6383061892909 kg.CO2eq/year
[codecarbon INFO @ 22:37:34] Energy consumed for

Epoch [30/30], Loss: 1.0495

--- Model Analysis ---
Parameter Count: 128807306
Model Size: 491.37 MB
FLOPs: 0.27 GFLOPs
Training Time: 2236.22 seconds
Average Inference Time: 3.491727 ms

--- Energy and Emissions Report ---
CO2 Emissions (CodeCarbon): 0.018779 kg
Average GPU Power Consumption: 68.28 W
Total GPU Energy Consumption: 42.410667 kWh


In [7]:
student_model.eval()
correct_test = 0
total_test = 0
with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = student_model(images)
        _, predicted = torch.max(outputs, 1)
        total_test += labels.size(0)
        correct_test += (predicted == labels).sum().item()

test_accuracy = 100 * correct_test / total_test
print(f"Student Model Test Accuracy: {test_accuracy:.2f}%")

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:617: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


Student Model Test Accuracy: 83.18%
